# <font color=blue> <u>  HomeWork 1</u> </font>
<br>

## ID1:205914609

## ID2:207906884


## <u> Classification Tree </u>

Import relevant packages

##  Section A+B:

In [1]:
from typing import List
import pandas as pd
import numpy as np
from datetime import datetime
import math
import random
import bisect
import os


    
Function to calculate Gini Index

In [2]:
def gini_index(classffier_arr: np.ndarray):
    # determine which unique values the classffier_arry have and count how many from each class
    values, counts = np.unique(classffier_arr, return_counts=True)
    # calculate how many values all and all in the classfier
    S = len(classffier_arr)
    # calculate gini index equation
    gini_idx = 1 - ((counts / S) ** 2).sum()

    return gini_idx

Function to calculate the total Gini of the split

In [3]:
def total_gini(attr_array: np.ndarray, splitter: float, classffier_arr: np.ndarray):
    # store all values above the splitter value
    above_splitter = classffier_arr[attr_array >= splitter]
    # store all values below the splitter value
    below_splitter = classffier_arr[attr_array < splitter]

    # calculate gini index for both groups
    above_splitter_gini = gini_index(classffier_arr=above_splitter)
    below_splitter_gini = gini_index(classffier_arr=below_splitter)
    # calculate how many values in the column
    S = len(classffier_arr)
    # calculate the totla gini of the column, based on the current splitter
    total = (len(above_splitter) / S) * above_splitter_gini + (len(below_splitter) / S) * below_splitter_gini

    return total

Function to calculate the best Gini Index for an attribute
<br>
Returns the best Gini and the best splitter of the obserations to get this Gini

In [4]:
def best_gini_for_attr(attr_array, classfier_arr):
    # store the unique values of the data in a sorted array
    unique_attr_array = np.unique(attr_array)
    min_gini = 1
    min_splitter = 0

    # for every mean between 2 adjacent values, calculate total gini
    for i in range((len(unique_attr_array) - 1)):
        splitter = float(np.mean(unique_attr_array[i:i + 2]))
        temp_gini = total_gini(attr_array, splitter, classfier_arr)

        # if its the best total gini store it as min_gini
        if temp_gini < min_gini:
            min_gini = temp_gini
            min_splitter = splitter

    return min_gini, min_splitter

Function to choose the best attribute to split by the data
<br>
The attribute with the minimum Gini index will be chosen
<br>
The function returns:
<br>
min_gini- the minimum Gini of the attribute to split by
<br>
splitter-the value to split by the data
<br>
best_col-the name to the attribute to split the data by

In [5]:
def choose_column_to_split_by(data_df: pd.DataFrame, classffier_arr: np.ndarray):
    best_col = ''
    splitter = 0
    min_gini = 1.1
    # for every attribute in the data frame
    for column in data_df.columns.values:
        # calculate the best gini for the current attribute
        temp_gini, temp_splitter = best_gini_for_attr(data_df[column].values, classfier_arr=classffier_arr)
        # if it is the best total gini-store it as min gini,
        # best splitter for this attribute and the attribute's name
        if temp_gini < min_gini:
            min_gini = temp_gini
            splitter = temp_splitter
            best_col = column

    return min_gini, splitter, best_col


Class Node: defining the nodes of the tree

In [6]:
class Node:
    def __init__(
            self,
            data_df: pd.DataFrame,
            classffier_arr: np.ndarray,
            depth
    ):
        self.data_df = data_df
        self.classffier_arr = classffier_arr
        self.depth = depth
        self.column = None
        self.gini = None
        self.splitter = None

        # array for left and right children
        self.children = []
        # determine which unique values the classffier_arry have and count how many from each class
        values, counts = np.unique(self.classffier_arr, return_counts=True)
        # store and array of the unique values in the classfier
        self.values_names = values
        # store the count of each class
        self.values = counts
        # store the class with the most values
        self.cls = values[np.argmax(counts)]

    # if all the data is form 1 class
    def perfectly_classified(self):
        return len(self.values) == 1

    # Function to predict the class of an observation in the data
    # gets only one row from the data
    def classify(self, data_row):
        # if the node is a leaf than classfiy
        # the current row as the majority class of the current node
        if len(self.children) == 0:
            return self.cls
        # store the current value of the column we are currently splitting by
        current_value = data_row[self.column]

        # if the current value is smaller than the node's splitter
        # go to the left child to classify
        # return the result of the left child
        if current_value < self.splitter and self.children[0] is not None:
            return self.children[0].classify(data_row)
        # if the current value is bigger or equals to the node's splitter
        # go to the right child to classify
        # return the result of the right child
        elif current_value >= self.splitter and self.children[1] is not None:
            return self.children[1].classify(data_row)
        else:
            # return the class of this node
            return self.cls

Class dt_tree: the class of the classification tree

In [7]:
class dt_tree:
    def __init__(self, depth
                 ):
        self.root = None
        self.depth = depth

    def dt_trainer(self, data_df: pd.DataFrame, classffier_arr: np.ndarray):
        # Initialize a queue for the nodes of the tree
        q_nodes: List[Node] = []
        # create root node
        root_node = Node(data_df=data_df, classffier_arr=classffier_arr, depth=0)
        # add root node to the queue
        q_nodes.append(root_node)
        # as long as we have nodes in the queue
        while len(q_nodes) > 0:
            # get first node in the queue
            node = q_nodes.pop(0)
            # check if the node is a leaf(contains only one class) or data frame is empty
            if node.perfectly_classified() or len(node.data_df.columns) == 0 or node.depth == self.depth:
                # calculate gini score for the current data
                node.gini = gini_index(node.classffier_arr)
                # continue to the next node
                continue
            # calculate which attribute to split by-the one with the best gini will be chosen
            min_gini, splitter, best_col = choose_column_to_split_by(data_df=node.data_df,
                                                                     classffier_arr=node.classffier_arr)

            # store the values of the attribute to split data by
            node.column = best_col
            node.gini = min_gini
            node.splitter = splitter

            # Store the data that has bigger value from the splitter
            # and drop the current attribute from the data
            left_node_df = node.data_df[node.data_df[best_col] < splitter]
            left_node = None
            # if exists data for the left node(bigger than the splitter)
            if len(left_node_df) > 0:
                # create left node and store the data in it
                left_node = Node(
                    data_df=left_node_df,
                    classffier_arr=node.classffier_arr[node.data_df[best_col] < splitter],
                    depth=node.depth + 1
                )
                # add the new node to the queue
                q_nodes.append(left_node)

            # Store the  data that has smaller value from the splitter
            # and drop the current attribute from the data
            right_node_df = node.data_df[node.data_df[best_col] >= splitter]
            right_node = None
            # if exists data for the right node(smaller than the splitter)
            if len(right_node_df) > 0:
                # create right node and store the data in it
                right_node = Node(
                    data_df=node.data_df[node.data_df[best_col] >= splitter],
                    classffier_arr=node.classffier_arr[node.data_df[best_col] >= splitter],
                    depth=node.depth + 1
                )
                # add the new node to the queue
                q_nodes.append(right_node)
            # store the left and right new nodes to the current node's children array
            node.children = [left_node, right_node]

        return root_node

    def dt_inference(self, dt: Node, data_df: pd.DataFrame, calssifier_arr):
        correct = 0
        # as long as we have rows in the data frame
        for i in range(len(data_df)):
            row = data_df.iloc[i]
            # predict the class of the current row according to the tree's decision
            predicted = dt.classify(row)
            # store the real class of the current row
            real = calssifier_arr[i]

            # if the predition was corret- count it
            if real == predicted:
                correct += 1

        # calculate how many true predictions where made from all the data
        accuracy = correct / len(data_df)
        return accuracy

    def dt_results(self, dt: Node, data_df: pd.DataFrame):
        results_arr = []

        # as long as we have rows in the data frame
        for i in range(len(data_df)):
            row = data_df.iloc[i]
            # predict the class of the current row according to the tree's decision
            predicted = dt.classify(row)
            results_arr.append(predicted)

        return results_arr


Function to choose the tree with the best accuracy

In [8]:
def best_class_tree(class_tree_A, class_tree_B, class_tree_C, accA, accB, accC):
    if accA >= accB and accA >= accC:
        return class_tree_A, accA
    if accB >= accA and accB >= accC:
        return class_tree_B, accB
    else:
        return class_tree_C, accC

Train, validate and test the tree

In [9]:
# get the path to the folder of the script
folder_path = os.getcwd()
# store csv file name as a string
csv_name = "data.csv"
# create the path and store it
csv_path = os.path.join(folder_path, csv_name)
# Read data from csv file and store in real_estate
real_estate = pd.read_csv(csv_path, index_col=[0])
                             
# store area_type column as classfier
classfier_arr = np.array(real_estate['area_type'])
 # store data without the classfier
real_estate_features = real_estate.drop('area_type', axis=1)
# store train data
train_df = real_estate_features.iloc[:8040]
# store the classfier for train data
train_cls_arr = classfier_arr[:8040]
# store the validation data
validation_df = real_estate_features.iloc[8041:10050]
# store the validation classifer
validation_cls_arr = classfier_arr[8041:10050]
print("Validation:")
##############trail 1 with depth 5 ##########
start_time_A = datetime.now()
# build decision tree with depth 5
class_tree_A = dt_tree(5)
class_tree_A.root = class_tree_A.dt_trainer(train_df, train_cls_arr)
print("Tree with", class_tree_A.depth, "was built successfully!")
accA = class_tree_A.dt_inference(class_tree_A.root, validation_df, validation_cls_arr)
print("The accuracy of the tree is:", accA)
end_time_A = datetime.now()
##########################################################

##############trail 2 with depth 10 ##########
start_time_B = datetime.now()
# build decision tree with depth 10
class_tree_B = dt_tree(10)
class_tree_B.root = class_tree_B.dt_trainer(train_df, train_cls_arr)
print("Tree with", class_tree_B.depth, "was built successfully!")
accB = class_tree_B.dt_inference(class_tree_B.root, validation_df, validation_cls_arr)
print("The accuracy of the tree is:", accB)
end_time_B = datetime.now()
##########################################################

##############trail 3 with depth 15 ##########
start_time_C = datetime.now()
# build decision tree with depth 15
class_tree_C = dt_tree(15)
class_tree_C.root = class_tree_C.dt_trainer(train_df, train_cls_arr)
print("Tree with", class_tree_C.depth, "was built successfully!")
accC = class_tree_C.dt_inference(class_tree_C.root, validation_df, validation_cls_arr)
print("The accuracy of the tree is:", accC)
end_time_C = datetime.now()
##########################################################
#check which tree has the best accuracy
best_tree, best_accuracy = best_class_tree(class_tree_A, class_tree_B, class_tree_C, accA, accB, accC)
best_accuracy = f"{best_accuracy:.3%}"
print("The best Classification Tree has", best_tree.depth, "depth, with accuracy:", best_accuracy)
best_runtime=None
if best_tree == class_tree_A:
    print('run time: {}'.format(end_time_A - start_time_A))
    best_runtime=end_time_A - start_time_A
if best_tree == class_tree_B:
    print('run time: {}'.format(end_time_B - start_time_B))
    best_runtime=end_time_B - start_time_B
if best_tree == class_tree_C:
    print('run time: {}'.format(end_time_C - start_time_C))
    best_runtime=end_time_C - start_time_C

# store the test data
test_df = real_estate_features.iloc[10051:]
# store the test classifer
test_cls_arr = classfier_arr[10051:]
# calculate accuracy of the test data based on the decision tree
test_acc = best_tree.dt_inference(best_tree.root, test_df, test_cls_arr)
test_acc = f"{test_acc:.3%}"
print("Test:")
print("The accuracy of the tree is:", test_acc)
# store test results
test_results = best_tree.dt_results(best_tree.root, test_df)

Validation:
Tree with 5 was built successfully!
The accuracy of the tree is: 0.9039323046291687
Tree with 10 was built successfully!
The accuracy of the tree is: 0.9069188651070185
Tree with 15 was built successfully!
The accuracy of the tree is: 0.9024390243902439
The best Classification Tree has 10 depth, with accuracy: 90.692%
run time: 0:01:40.858751
Test:
The accuracy of the tree is: 90.804%


##  Section C:

Import relevant packages from sklearn to build a classification tree and regression tree

In [10]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
# model building
from sklearn.model_selection import train_test_split, GridSearchCV
# model evaluation
from sklearn.metrics import accuracy_score, mean_squared_error

define the model

In [11]:
# define model
dt = DecisionTreeClassifier()

# define parameter grid
parameters_grid = {
    'max_depth': [5, 10, 15]
}

# define grid search
grid_search = GridSearchCV(estimator=dt, param_grid=parameters_grid)

fit the model

In [12]:
# fit estimator
sk_class_tree_start=datetime.now()
grid_search.fit(train_df, train_cls_arr)
sk_class_tree_end=datetime.now()

# get best estimator
best = grid_search.best_estimator_
#print best max_depth
print(best)

DecisionTreeClassifier(max_depth=10)


predict the area type for validation data

In [13]:
# predict
predictions = best.predict(validation_df)

calculate accuracy for validation data and print it+compare to the accuracy of the presonal decision tree classifier

In [14]:
#calculate accuracy for validation data
acc = round(accuracy_score(validation_cls_arr,predictions), 3)

df = pd.DataFrame([acc])
df = df.rename(index={0: 'Decision Tree Classifier'}, columns={0: 'Accuracy'})
print("sklrean tree:")
print(df)
print('run time: {}'.format(sk_class_tree_end-sk_class_tree_start))
print("personal tree:")
print(best_accuracy)
print('run time: {}'.format(best_runtime))

sklrean tree:
                          Accuracy
Decision Tree Classifier     0.911
run time: 0:00:00.763864
personal tree:
90.692%
run time: 0:01:40.858751


predict the area type on the test data

In [15]:
# predict
predictions = best.predict(test_df)

calculate accuracy for test data and print it+compare to the accuracy of the presonal decision tree classifier

In [16]:
#calculate accuracy for validation data
acc = round(accuracy_score(test_cls_arr,predictions), 3)

df = pd.DataFrame([acc])
df = df.rename(index={0: 'Decision Tree Classifier'}, columns={0: 'Accuracy'})
print("sklrean tree:")
print(df)
print("personal tree:")
print(test_acc)


sklrean tree:
                          Accuracy
Decision Tree Classifier     0.911
personal tree:
90.804%


Our decision tree classifier has a very close accuracy to sklrean decision tree classifier,
but the sklearn algorithm is much faster.
We estimate the reason for that is that we used some while and for loops, while sklearn used more efficient methods we could not think of.
In addition they use gridsearch, which makes their algorithm more accurate and much faster.

## <u> Regression Tree </u>

## Section A+B:

Function to calculate the SSR score of an attribute

In [17]:
def ssr_score(attr_array: np.ndarray, splitter: float, classfier_arr: np.ndarray):
    # if the data frame is empty, then the sum is 0
    if len(classfier_arr[attr_array >= splitter]) == 0:
        sum_above_splitter = 0
    else:
        # calculate the mean of all values above the splitter value
        mean_above_splitter = classfier_arr[attr_array >= splitter].mean()
        # sum the square residuals(real attribute value-mean of above split values) of all elements in the array
        # that are bigger than the splitter
        sum_above_splitter = ((classfier_arr[attr_array >= splitter] - mean_above_splitter) ** 2).sum()
    # if the data frame is empty, then the sum is 0
    if len(classfier_arr[attr_array < splitter]) == 0:
        sum_below_splitter = 0
    else:
        # calculate the mean of below the splitter value
        mean_below_splitter = classfier_arr[attr_array < splitter].mean()
        # sum the square residuals(real attribute value-mean of below split values) of all elements in the array
        # that are smaller than the splitter
        sum_below_splitter = ((classfier_arr[attr_array < splitter] - mean_below_splitter) ** 2).sum()

    # return SSR score for this split
    return sum_above_splitter + sum_below_splitter

Function to calculate the best SSR score for an attribute
Returns the best SSR and the best splitter of the obserations to get this SSR

In [18]:
def best_ssr_for_attr(attr_array, classfier_arr):
    # store the unique values of the data in a sorted array
    unique_attr_array = np.unique(attr_array)
    min_ssr = None
    min_splitter = None

    if len(unique_attr_array) == 1:
        return ssr_score(attr_array, unique_attr_array[0], classfier_arr), unique_attr_array[0]

    # for every mean between 2 adjacent values, calculate ssr
    for i in range((len(unique_attr_array) - 1)):
        temp_splitter = float(np.mean(unique_attr_array[i:i + 2]))
        temp_ssr = ssr_score(attr_array, temp_splitter, classfier_arr)

        # if it is the first ssr calculated-store it as min ssr
        if min_ssr is None:
            min_ssr = temp_ssr
            min_splitter = temp_splitter
        # if its ssr store it as min_ssr
        if temp_ssr < min_ssr:
            min_ssr = temp_ssr
            min_splitter = temp_splitter

    return min_ssr, min_splitter


Function to choose the best attribute to split by the data
<br>
The attribute with the minimum SSR will be chosen
<br>
The function returns:
<br>
min_ssr- the minimum SSR of the attribute to split by
<br>
splitter-the value to split by the data
<br>
best_col-the name to the attribute to split the data by

In [19]:
def choose_column_to_split_by(data_df: pd.DataFrame, classffier_arr: np.ndarray):
    best_col = ''
    splitter = 0
    min_ssr = None
    # for every attribute in the data frame
    for column in data_df.columns.values:
        # calculate the best ssr for the current attribute
        temp_ssr, temp_splitter = best_ssr_for_attr(data_df[column].values, classfier_arr=classffier_arr)
        # if it is the best ssr-store it as min ssr,
        # best splitter for this attribute and the attribute's name
        if min_ssr is None:
            min_ssr = temp_ssr
            splitter = temp_splitter
            best_col = column

        if temp_ssr < min_ssr:
            min_ssr = temp_ssr
            splitter = temp_splitter
            best_col = column
    return min_ssr, splitter, best_col


Class Node: defining the nodes of the tree

In [20]:
class Node:
    def __init__(
            self,
            data_df: pd.DataFrame,
            classffier_arr: np.ndarray,
            depth
    ):
        self.data_df = data_df
        self.classffier_arr = classffier_arr
        self.depth = depth
        self.column = None
        self.ssr = None
        self.splitter = None
        # array for left and right children
        self.children = []
        self.mean_price = self.classffier_arr.mean()

    # gets only one row from the data
    def classify(self, data_row):
        # if the node is a leaf than classfiy
        # the current row as the majority class of the current node
        if len(self.children) == 0:
            return self.mean_price
        # store the current value of the column we are currently splitting by
        current_value = data_row[self.column]

        # if the current value is smaller than the node's splitter
        # go to the left child to classify
        # return the result of the left child
        if current_value < self.splitter and self.children[0] is not None:
            return self.children[0].classify(data_row)
        # if the current value is bigger or equal than the node's splitter
        # go to the right child to classify
        # return the result of the right child
        elif current_value >= self.splitter and self.children[1] is not None:
            return self.children[1].classify(data_row)
        else:
            # return the class of this node
            return self.mean_price

Class Regression_tree: defining the Regressio tree

In [21]:
class Regression_tree:
    def __init__(self,
                 min_split_size, depth):
        self.depth = depth
        self.min_split_size = min_split_size
        self.root = None

    def dt_trainer(self, data_df: pd.DataFrame, classffier_arr: np.ndarray):
        # Initialize a queue for the nodes of the tree
        q_nodes: List[Node] = []
        # create root node
        self.root = Node(data_df=data_df, classffier_arr=classffier_arr, depth=0)
        # add root node to the queue
        q_nodes.append(self.root)
        # as long as we have nodes in the queue
        while len(q_nodes) > 0:
            # get first node in the queue
            node = q_nodes.pop(0)

            # check if the node is a leaf or data frame is empty
            if len(node.data_df) < self.min_split_size or node.depth == self.depth:
                continue
            # calculate which attribute to split by-the one with the best ssr will be chosen
            min_ssr, splitter, best_col = choose_column_to_split_by(data_df=node.data_df,
                                                                    classffier_arr=node.classffier_arr)

            # store the values of the attribute to split data by
            node.column = best_col
            node.ssr = min_ssr
            node.splitter = splitter

            # Store the data that has bigger value from the splitter
            # and drop the current attribute from the data

            left_node_df = node.data_df[node.data_df[best_col] < splitter]
            left_node = None
            # if exists data for the left node(bigger than the splitter)
            if len(left_node_df) > 0:
                # create left node and store the data in it
                left_node = Node(
                    data_df=left_node_df,
                    classffier_arr=node.classffier_arr[node.data_df[best_col] < splitter],
                    depth=node.depth + 1
                )
                # add the new node to the queue
                q_nodes.append(left_node)

            # Store the  data that has smaller value from the splitter
            # and drop the current attribute from the data
            right_node_df = node.data_df[node.data_df[best_col] >= splitter]
            right_node = None
            # if exists data for the right node(smaller than the splitter)
            if len(right_node_df) > 0:
                # create right node and store the data in it
                right_node = Node(
                    data_df=node.data_df[node.data_df[best_col] >= splitter],
                    classffier_arr=node.classffier_arr[node.data_df[best_col] >= splitter],
                    depth=node.depth + 1

                )
                # add the new node to the queue
                q_nodes.append(right_node)
                # store the left and right new nodes to the current node's children array
            node.children = [left_node, right_node]

    def dt_inference(self, dt: Node, data_df: pd.DataFrame, calssifier_arr):
        sum = 0
        # as long as we have rows in the data frame
        for i in range(len(data_df)):
            row = data_df.iloc[i]
            # predict the price of the current row according to the tree's decision
            predicted = dt.classify(row)
            # calculate the difference between the real price and the predicted price
            # and square it
            sqr_residual = (calssifier_arr[i] - predicted) ** 2
            sum += sqr_residual
        # calculate mse by divide the total differences sum in the number of observations
        mse = sum / len(data_df)
        # calculate how many true predictions where made from all the data

        return mse

    def dt_results(self, dt: Node, data_df: pd.DataFrame):
        results_arr = []

        # as long as we have rows in the data frame
        for i in range(len(data_df)):
            row = data_df.iloc[i]
            # predict the price of the current row according to the tree's decision
            predicted = dt.classify(row)
            results_arr.append(predicted)

        return results_arr


Function to find the best regression tree-with the lowest mse

In [22]:
def best_reg_tree(reg_tree_A, reg_tree_B, reg_tree_C, mseA, mseB, mseC):
    if mseA <= mseB and mseA <= mseC:
        return reg_tree_A, mseA
    if mseB <= mseA and mseB <= mseC:
        return reg_tree_B, mseB
    else:
        return reg_tree_C, mseC

Train run and test the regression tree

In [23]:
# get the path to the folder of the script
folder_path = os.getcwd()
# store csv file name as a string
csv_name = "data.csv"
# create the path and store it
csv_path = os.path.join(folder_path, csv_name)
# Read data from csv file and store in real_estate
real_estate = pd.read_csv(csv_path, index_col=[0])

# store area_type column as classifier
classfier_arr = np.array(real_estate['price in rupees'])
# store data without the classifier
real_estate_features = real_estate.drop('price in rupees', axis=1)
real_estate_features.loc[(real_estate_features.area_type == 'B'), "area_type"] = 0
real_estate_features.loc[(real_estate_features.area_type == 'P'), "area_type"] = 1
real_estate_features['area_type'] = pd.to_numeric(real_estate_features['area_type'])
# store train data
train_df = real_estate_features.iloc[:8040]
# store the classifier for train data
train_price_arr = classfier_arr[:8040]
# store the validation data
validation_df = real_estate_features.iloc[8041:10050]
# store the validation classifer
validation_cls_arr = classfier_arr[8041:10050]

print("Validation:")
###########trail 1 with minimum 3 observations and maximu depth=3
# build decision tree based on training data
start_time_A = datetime.now()
reg_tree_A = Regression_tree(3, 3)
reg_tree_A.dt_trainer(train_df, train_price_arr)
print("Tree with", reg_tree_A.depth, "was built successfully!")
mseA = reg_tree_A.dt_inference(reg_tree_A.root, validation_df, validation_cls_arr)
print("The MSE of the tree is:", mseA)
end_time_A = datetime.now()
##########################################################

###########trail 2 with minimum 3 observations and maximu depth=6
# build decision tree based on training data
start_time_B = datetime.now()
reg_tree_B = Regression_tree(3, 6)
# store the validation data
reg_tree_B.dt_trainer(train_df, train_price_arr)
print("Tree with", reg_tree_B.depth, "was built successfully!")
mseB = reg_tree_B.dt_inference(reg_tree_B.root, validation_df, validation_cls_arr)
print("The MSE of the tree is:", mseB)
end_time_B = datetime.now()
##########################################################

###########trail 3 with minimum 3 observations and maximu depth=9
# build decision tree based on training data
start_time_C = datetime.now()
reg_tree_C = Regression_tree(3, 9)
# store the validation data
reg_tree_C.dt_trainer(train_df, train_price_arr)
print("Tree with", reg_tree_C.depth, "was built successfully!")
mseC = reg_tree_C.dt_inference(reg_tree_C.root, validation_df, validation_cls_arr)
print("The MSE of the tree is:", mseC)
end_time_C = datetime.now()
##########################################################

best_tree, best_mse = best_reg_tree(reg_tree_A, reg_tree_B, reg_tree_C, mseA, mseB, mseC)
print("The best regression Tree is has", best_tree.depth, "depth, with mse:", best_mse)
if best_tree == reg_tree_A:
    print('run time: {}'.format(end_time_A - start_time_A))
    best_runtime=end_time_A - start_time_A
if best_tree == reg_tree_B:
    print('run time: {}'.format(end_time_B - start_time_B))
    best_runtime=end_time_B - start_time_B
if best_tree == reg_tree_C:
    print('run time: {}'.format(end_time_C - start_time_C))
    best_runtime=end_time_C - start_time_C

print("Test:")
validation_results = best_tree.dt_results(best_tree.root, validation_df)
# store the test data
test_df = real_estate_features.iloc[10051:]
# store the test classifer
test_price_arr = classfier_arr[10051:]
test_mse = best_tree.dt_inference(best_tree.root, test_df, test_price_arr)
print("The MSE of the tree is:", test_mse)
test_results = best_tree.dt_results(best_tree.root, test_df)

Validation:
Tree with 3 was built successfully!
The MSE of the tree is: 58316389500846.53
Tree with 6 was built successfully!
The MSE of the tree is: 49252961604795.86
Tree with 9 was built successfully!
The MSE of the tree is: 57069055678763.38
The best regression Tree is has 6 depth, with mse: 49252961604795.86
run time: 0:00:05.825498
Test:
The MSE of the tree is: 67386316078160.95


## Section c:

define the model

In [24]:
# define model
dt = DecisionTreeRegressor()

# define parameter grid
parameters_grid = {
    'max_depth': [3, 6, 9],
    'min_samples_leaf': [20,2,3]
}

# define grid search
grid_search = GridSearchCV(estimator=dt, param_grid=parameters_grid)

fit the model

In [25]:
# fit estimator
sk_reg_tree_start=datetime.now()
grid_search.fit(train_df, train_price_arr)
sk_reg_tree_end=datetime.now()

# get best estimator
best = grid_search.best_estimator_
#print best max_depth
print(best)

DecisionTreeRegressor(max_depth=9, min_samples_leaf=3)


predict the area type for validation data

In [26]:
# predict
predictions = best.predict(validation_df)

calculate mse for validation data and print it+compare to the mse of the presonal regression tree

In [27]:
#calculate accuracy for validation data
MSE = round(mean_squared_error(validation_cls_arr,predictions), 3)

df = pd.DataFrame([MSE])
df = df.rename(index={0: 'Decision Tree Regressor'}, columns={0: 'MSE'})
print("sklrean tree:")
print(df)
print('run time: {}'.format(sk_reg_tree_end-sk_reg_tree_start))
print("personal tree with depth:",best_tree.depth)
print('MSE:',best_mse)
print('run time: {}'.format(best_runtime))

sklrean tree:
                                  MSE
Decision Tree Regressor  4.468983e+13
run time: 0:00:01.094749
personal tree with depth: 6
MSE: 49252961604795.86
run time: 0:00:05.825498


predict the area type for test data

In [28]:
# predict
predictions = best.predict(test_df)

calculate mse for the test data and print it+compare to the mse of the presonal regression tree

In [29]:
#calculate accuracy for validation data
MSE = round(mean_squared_error(test_price_arr,predictions), 3)

df = pd.DataFrame([MSE])
df = df.rename(index={0: 'Decision Tree Regressor'}, columns={0: 'MSE'})
print("sklrean tree:")
print(df)
print("personal tree with depth:",best_tree.depth)
print('MSE:',test_mse)


sklrean tree:
                                  MSE
Decision Tree Regressor  6.027681e+13
personal tree with depth: 6
MSE: 67386316078160.95


Sklearn algorithm for regression tree has 12% lower MSE from our regression tree algorithm.
In addition sklearn algorith is much more faster than our algorithm.
We estimate the reason for that is that we used some while and for loops, while sklearn used more efficient methods we could not think of.
In addition they use gridsearch to tune the best hyper-parameters, while we don't, which makes their algorithm more accurate and much faster.

## Adaboost classifier


Function to calculate Gini Index

In [47]:
def gini_index(classffier_arr: np.ndarray):
    # determine which unique values the classffier_arry have and count how many from each class
    values, counts = np.unique(classffier_arr, return_counts=True)
    # calculate how many values all and all in the classfier
    S = len(classffier_arr)
    # calculate gini index equation
    gini_idx = 1 - ((counts / S) ** 2).sum()

    return gini_idx


Function to calculate the total Gini of the split

In [48]:
def total_gini(attr_array: np.ndarray, splitter: float, classffier_arr: np.ndarray):
    # store all values above the splitter value
    above_splitter = classffier_arr[attr_array >= splitter]
    # store all values below the splitter value
    below_splitter = classffier_arr[attr_array < splitter]

    # calculate gini index for both groups
    above_splitter_gini = gini_index(classffier_arr=above_splitter)
    below_splitter_gini = gini_index(classffier_arr=below_splitter)
    # calculate how many values in the column
    S = len(classffier_arr)
    # calculate the totla gini of the column, based on the current splitter
    total = (len(above_splitter) / S) * above_splitter_gini + (len(below_splitter) / S) * below_splitter_gini

    return total

Function to calculate the best Gini Index for an attribute
<br>
Returns the best Gini and the best splitter of the obserations to get this Gini

In [49]:
def best_gini_for_attr(attr_array, classfier_arr):
    # store the unique values of the data in a sorted array
    unique_attr_array = np.unique(attr_array)
    min_gini = 1
    min_splitter = 0

    # for every mean between 2 adjacent values, calculate total gini
    for i in range((len(unique_attr_array) - 1)):
        splitter = float(np.mean(unique_attr_array[i:i + 2]))
        temp_gini = total_gini(attr_array, splitter, classfier_arr)

        # if its the best total gini store it as min_gini
        if temp_gini < min_gini:
            min_gini = temp_gini
            min_splitter = splitter

    return min_gini, min_splitter

Function to choose the best attribute to split by the data
<br>
The attribute with the minimum Gini index will be chosen
<br>
The function returns:
<br>
min_gini- the minimum Gini of the attribute to split by
<br>
splitter-the value to split by the data
<br>
best_col-the name to the attribute to split the data by

In [50]:
def choose_column_to_split_by(data_df: pd.DataFrame, classffier_arr: np.ndarray):
    best_col = ''
    splitter = 0
    min_gini = 1.1
    # for every attribute in the data frame
    for column in data_df.columns.values:
        # calculate the best gini for the current attribute
        temp_gini, temp_splitter = best_gini_for_attr(data_df[column].values, classfier_arr=classffier_arr)
        # if it is the best total gini-store it as min gini,
        # the best splitter for this attribute and the attribute's name
        if temp_gini < min_gini:
            min_gini = temp_gini
            splitter = temp_splitter
            best_col = column

    return min_gini, splitter, best_col


Class Node: defining the nodes of the tree

In [51]:
class Node:
    def __init__(
            self,
            data_df: pd.DataFrame,
            classffier_arr: np.ndarray,
            depth
    ):
        self.data_df = data_df
        self.classffier_arr = classffier_arr
        self.depth = depth
        self.column = None
        self.gini = None
        self.splitter = None
        # array for left and right children
        self.children = []
        # determine which unique values the classffier_arry have and count how many from each class
        values, counts = np.unique(self.classffier_arr, return_counts=True)
        # store and array of the unique values in the classfier
        self.values_names = values
        # store the count of each class
        self.values = counts
        # store the class with the most values
        self.cls = values[np.argmax(counts)]

    # if all the data is form 1 class
    def perfectly_classified(self):
        return len(self.values) == 1

    # gets only one row from the data
    def classify(self, data_row):
        # if the node is a leaf than classify
        # the current row as the majority class of the current node
        if len(self.children) == 0:
            return self.cls
        # store the current value of the column we are currently splitting by
        current_value = data_row[self.column]

        # if the current value is smaller than the node's splitter
        # go to the left child to classify
        # return the result of the left child
        if current_value < self.splitter and self.children[0] is not None:
            return self.children[0].classify(data_row)
        # if the current value is bigger or equals than the node's splitter
        # go to the right child to classify
        # return the result of the right child
        elif current_value >= self.splitter and self.children[1] is not None:
            return self.children[1].classify(data_row)
        else:
            # return the class of this node
            return self.cls


Defina a class of a stump

In [52]:
class Stump:
    def __init__(
            self, data_df: pd.DataFrame,
            classffier_arr: np.ndarray,
    ):
        self.data_df = data_df
        self.classffier_arr = classffier_arr
        self.childern = []
        self.root = None
        # stores the weight for each observation in the data frame
        self.weight_array = np.array([])
        # stores the stump's weight
        self.weight = None

    # function to fill the stumps with it's root and 2 leafs
    # and his properties
    def stump_trainer(self, data_df: pd.DataFrame, classffier_arr: np.ndarray):
        # create root node
        root_node = Node(data_df=data_df, classffier_arr=classffier_arr, depth=0)
        # calculate which attribute to split by-the one with the best gini will be chosen
        min_gini, splitter, best_col = choose_column_to_split_by(data_df=root_node.data_df,
                                                                 classffier_arr=root_node.classffier_arr)
        # store the values of the attribute to split data by
        root_node.column = best_col
        root_node.gini = min_gini
        root_node.splitter = splitter

        # The start weight for every observation will be
        # 1/the number of the observations in the data frame
        start_weight = 1 / len(data_df)
        # store in weight_array the start weight for every observation
        self.weight_array = np.full(len(data_df), start_weight)

        # Store the data that has smaller value from the splitter
        # and drop the current attribute from the data
        left_node_df = root_node.data_df[root_node.data_df[best_col] < splitter]
        left_node = None
        # if exists data for the left node(bigger than the splitter)
        if len(left_node_df) > 0:
            # create left node and store the data in it
            left_node = Node(
                data_df=left_node_df,
                classffier_arr=root_node.classffier_arr[root_node.data_df[best_col] < splitter],
                depth=root_node.depth + 1
            )

        # Store the  data that has bigger or equal value from the splitter
        right_node_df = root_node.data_df[root_node.data_df[best_col] >= splitter]
        right_node = None
        # if exists data for the right node(smaller than the splitter)
        if len(right_node_df) > 0:
            # create right node and store the data in it
            right_node = Node(
                data_df=root_node.data_df[root_node.data_df[best_col] >= splitter],
                classffier_arr=root_node.classffier_arr[root_node.data_df[best_col] >= splitter],
                depth=root_node.depth + 1
            )
        # store the left and right new nodes to the current node's children array
        root_node.children = [left_node, right_node]
        return root_node

    # function to calculate the total error of the stump
    def total_error(self):
        # initialize the number of errors to 0
        error = 0
        # store the stump's data frame in data_df
        data_df = self.data_df
        # store the stump's classffier_arr in classfier_arr
        classifier_arr = self.classffier_arr
        # as long as we have rows in the data frame
        for i in range(len(data_df)):
            row = data_df.iloc[i]
            # predict the class of the current row according to the stump's decision
            predicted = self.root.classify(row)
            # store the real class of the current row
            real = classifier_arr[i]

            # if the predition was wrong- count it
            if real != predicted:
                error += 1
        # return the total error for this stump
        return error / len(data_df)

    # function to update the weight of the data frame's observations
    def change_weight(self):
        # store the stump's data frame in data_df
        data_df = self.data_df
        # store the stump's classffier_arr in classfier_arr
        classifier_arr = self.classffier_arr
        # store the new weight for an observation correctly classified
        # according to the equation: new_weight=sample_weight*e**(-stump_weight)
        right_weight = self.weight_array[0] * math.e ** (-self.weight)
        # store the new weight for an observation incorrectly classified
        # according to the equation: new_weight=sample_weight*e**stump_weight
        wrong_weight = self.weight_array[0] * math.e ** self.weight
        # as long as we have rows in the data frame
        for i in range(len(data_df)):
            row = data_df.iloc[i]
            # predict the class of the current row according to the stump's decision
            predicted = self.root.classify(row)
            # store the real class of the current row
            real = classifier_arr[i]

            # if the prediction was wrong- increase weight
            if real != predicted:
                self.weight_array[i] = wrong_weight
            # if the prediction was right-decrease the weight
            else:
                self.weight_array[i] = right_weight
            # calculate the sum of the weights of all observations
        sum_weight = self.weight_array.sum()
        # normalize the weight of every observation by
        # divding the current observation's weight by the sum of the weights of all observations
        for i in range(len(self.weight_array)):
            self.weight_array[i] = self.weight_array[i] / sum_weight

    # function to create a new dataframe and his classifer to create from them a new stump
    def create_weighted_df(self):
        # Intialize an array to store the weights ranges
        ranges_array = np.array([])
        weight_sum = 0
        # for every weight of an observation
        for curr_weight in self.weight_array:
            # add the weight of the current observation to weight_sum
            weight_sum = weight_sum + curr_weight
            # append the new sum to the weight ranges array
            ranges_array = np.append(ranges_array, weight_sum)
        # create a new data frame with the same columns as the given data frame
        new_df = pd.DataFrame(columns=self.data_df.columns)
        # create a new empty classfier
        new_classfier = np.array([])
        # for every observation in the data frame
        for i in range(len(self.data_df)):
            # use a random number between 0 and 1
            # and save the index that should be for the random number if it was in the
            # weights array
            index = bisect.bisect_right(ranges_array, random.random())
            # add the index of this row to the new data frame
            # this step will create a data frame with the observations that have more weight
            # more times
            new_df = new_df.append(self.data_df.iloc[index], ignore_index=True)
            # add the class of the selected observation to the classifier array
            new_classfier = np.append(new_classfier, self.classffier_arr[index])
        # return the new data frame and his classifier to create a new stump from them
        return new_df, new_classfier


Define a class of adaboost

In [53]:
class Adaboost:
    def __init__(
            self,
            max_stumps
    ):
        # hyper parameter for the number of stumps
        self.max_stumps = max_stumps
        # an array to store the stumps of the booster
        self.stumps_array = []

    # function to create the stumps of the booster and store them in the booster's stumps array
    def adaboost_trainer(self, data_df: pd.DataFrame, classffier_arr: np.ndarray):
        # create the selected number of stumps
        for i in range(1, self.max_stumps):
            # create a new empty stump
            temp_stump = Stump(data_df, classffier_arr)
            # Create the stump's root and nodes
            temp_stump.root = temp_stump.stump_trainer(data_df, classffier_arr)
            # calculate the total error of the stump and store it in the stump
            temp_error = temp_stump.total_error()
            # calculate the weight of the sutmp according to totla error and store it in the stump
            temp_stump.weight = 0.5 * math.log((1 - temp_error) / temp_error)
            # update the weight of every observation in the stump
            temp_stump.change_weight()
            # add the new stump to the booster's array of stumps
            self.stumps_array.append(temp_stump)
            # create a new data frame and classffier to create a new stump
            # the new data frame will contain the observations with bigger weight
            # more times
            data_df, classffier_arr = temp_stump.create_weighted_df()

    # function to predict the class of the row according to the weights of the stumps
    def predict(self, data_row):
        # the sum of weights of stumps that chose 'B' class
        btype_sum = 0
        # the sum of weights of stumps that chose 'P' class
        ptype_sum = 0
        # f or every stump in the booster
        for stump in self.stumps_array:
            # predict the class with the current stump
            predicted = stump.root.classify(data_row)
            # add the weight of the stump to the sum of the result class
            if predicted == 'B':
                btype_sum += stump.weight
            else:
                ptype_sum += stump.weight
        # predict the class of the observation according to the class that has more weight
        # from all of the stumps
        if btype_sum <= ptype_sum:
            return 'P'
        return 'B'

    # function to predict the class of every observation and calculate the accuracy of the booster
    def inference(self, data_df: pd.DataFrame, calssifier_arr):
        correct = 0
        # as long as we have rows in the data frame
        for i in range(len(data_df)):
            row = data_df.iloc[i]
            # predict the class of the current row according to the tree's decision
            predicted = self.predict(row)
            # store the real class of the current row
            real = calssifier_arr[i]

            # if the prediction was correct- count it
            if real == predicted:
                correct += 1

        # calculate how many true predictions where made from all the data
        accuracy = correct / len(data_df)
        return accuracy

    # function to return an array of the predictions of every observation
    def results(self, data_df: pd.DataFrame):
        results_arr = []
        # as long as we have rows in the data frame
        for i in range(len(data_df)):
            row = data_df.iloc[i]
            # predict the class of the current row according to the booster's decision
            predicted = self.predict(row)
            results_arr.append(predicted)

        return results_arr


Function to choose the best booster- with the best accuracy

In [54]:
def best_booster(booster_A, booster_B, booster_C, accA, accB, accC):
    if accA >= accB and accA >= accC:
        return booster_A, accA
    if accB >= accA and accB >= accC:
        return booster_B, accB
    else:
        return booster_C, accC


Train, validate and test the booster

In [56]:
# get the path to the folder of the script
folder_path = os.getcwd()
# store csv file name as a string
csv_name = "data.csv"
# create the path and store it
csv_path = os.path.join(folder_path, csv_name)
# Read data from csv file and store in real_estate
real_estate = pd.read_csv(csv_path, index_col=[0])
                             
# store area_type column as classfier
classfier_arr = np.array(real_estate['area_type'])
# store data without the classfier
real_estate_features = real_estate.drop('area_type', axis=1)
# store train data
train_df = real_estate_features.iloc[:8040]
# store the classfier for train data
train_cls_arr = classfier_arr[:8040]
# store the validation data
validation_df = real_estate_features.iloc[8041:10050]
# store the validation classifer
validation_cls_arr = classfier_arr[8041:10050]
print("Validation:")

##############trail 1 with 5 stumps ##########
start_time_A = datetime.now()
# build Booster with 5 stumps
booster_A = Adaboost(5)
booster_A.adaboost_trainer(train_df, train_cls_arr)
print("Booster with", booster_A.max_stumps, "stumps was built successfully!")
accA = booster_A.inference(validation_df, validation_cls_arr)
print("The accuracy of the Booster is:", accA)
end_time_A = datetime.now()
##########################################################

##############trail 2 with 8 stumps ##########
start_time_B = datetime.now()
# build Booster with 8 stumps
booster_B = Adaboost(8)
booster_B.adaboost_trainer(train_df, train_cls_arr)
print("Booster with", booster_B.max_stumps, "stumps was built successfully!")
accB = booster_B.inference(validation_df, validation_cls_arr)
print("The accuracy of the Booster is:", accA)
end_time_B = datetime.now()
##########################################################

##############trail 1 with 10 stumps ##########
start_time_C = datetime.now()
# build Booster with 10 stumps
booster_C = Adaboost(10)
booster_C.adaboost_trainer(train_df, train_cls_arr)
print("Booster with", booster_C.max_stumps, "stumps was built successfully!")
accC = booster_C.inference(validation_df, validation_cls_arr)
print("The accuracy of the Booster is:", accC)
end_time_C = datetime.now()
##########################################################

best_adaboost, best_accuracy = best_booster(booster_A, booster_B, booster_C, accA, accB, accC)
best_accuracy = f"{best_accuracy:.3%}"
print("The best Booster has", best_adaboost.max_stumps, "stumps, with accuracy:", best_accuracy)
if best_adaboost == booster_A:
    print('run time: {}'.format(end_time_A - start_time_A))
    best_runtime=end_time_A - start_time_A
if best_adaboost == booster_B:
    print('run time: {}'.format(end_time_B - start_time_B))
    best_runtime=end_time_B - start_time_B
if best_adaboost == booster_C:
    print('run time: {}'.format(end_time_C - start_time_C))
    best_runtime=end_time_C - start_time_C

validation_results = best_adaboost.results(validation_df)
# store the test data
test_df = real_estate_features.iloc[10051:]
# store the test classifer
test_cls_arr = classfier_arr[10051:]
# calculate accuracy of the test data based on the booster
best_accuracy = best_adaboost.inference(test_df, test_cls_arr)
best_accuracy = f"{best_accuracy:.3%}"
print("Test:")
print("The accuracy of the booster is:", best_accuracy)
# store test results
test_results = best_adaboost.results(test_df)

Validation:
Booster with 5 stumps was built successfully!
The accuracy of the Booster is: 0.8735689397710303
Booster with 8 stumps was built successfully!
The accuracy of the Booster is: 0.8735689397710303
Booster with 10 stumps was built successfully!
The accuracy of the Booster is: 0.8780487804878049
The best Booster has 10 stumps, with accuracy: 87.805%
run time: 0:02:23.501400
Test:
The accuracy of the booster is: 87.500%


## Section C

Import relevant libraries from sklearn

In [57]:
# adaboost
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor

# gradient boosting
from sklearn.ensemble import GradientBoostingClassifier

# model building
from sklearn.model_selection import train_test_split, GridSearchCV

# model evaluation
from sklearn.metrics import accuracy_score


define the model

In [58]:
# define model
ab = AdaBoostClassifier()

# define parameter grid
parameters_grid = {
    'n_estimators': [5, 8,10]
}

# define grid search
grid_search = GridSearchCV(estimator=ab, param_grid=parameters_grid)

fit the model

In [59]:
# fit estimator
adaboost_start_time=datetime.now()
grid_search.fit(validation_df, validation_cls_arr)
adaboost_end_time=datetime.now()

# get best estimator
best = grid_search.best_estimator_
pd.DataFrame.from_dict(grid_search.best_params_, orient='index', columns=['Selected Value'])

,Selected Value
n_estimators,10


predict the area type of the validation data

In [60]:
predictions=best.predict(validation_df)

calculate accuracy for validation data and print it+compare to the mse of the presonal regression tree

In [61]:
# calculate accuracy
acc = round(accuracy_score(validation_cls_arr, predictions), 3)

df = pd.DataFrame([acc])
df = df.rename(index={0: 'AdaBoost Classifier'}, columns={0: 'Accuracy'})
print("sklearn Adaboost:")
print(df)
print('run time: {}'.format(adaboost_end_time-adaboost_start_time))
print("personal booster with:",best_adaboost.max_stumps,"stumps:")
print('Accuracy:',best_accuracy)
print('run time: {}'.format(best_runtime))

sklearn Adaboost:
                     Accuracy
AdaBoost Classifier     0.884
run time: 0:00:00.816227
personal booster with: 10 stumps:
Accuracy: 87.500%
run time: 0:02:23.501400


predict the area type of the test data

In [62]:
predictions=best.predict(test_df)

calculate accuracy for test data and print it+compare to the accuracy of the presonal booster classifier

In [65]:
#calculate accuracy for validation data
acc = round(accuracy_score(test_cls_arr,predictions), 3)

df = pd.DataFrame([acc])
df = df.rename(index={0: 'Adaboost Classifier'}, columns={0: 'Accuracy'})
print("sklrean Adaboost:")
print(df)
print("personal Adaboost:")
print(best_accuracy)

sklrean Adaboost:
                     Accuracy
Adaboost Classifier     0.885
personal Adaboost:
87.500%


Our Adaboost is more accurate then sklearn's adaboost, but much slower.
We estimate the reason for that is that we used some for and while loops in our code, while sklearn used more efficient methods we could not think of.
In addition they use gridsearch to tune the best hyper-parameters, while we don't, which makes their algorithm more accurate and much faster